In [9]:
import numpy as np
import pandas as pd
import redis

In [10]:
r =redis.Redis(
    host='host_id_of_redis_db',
    port=port_no_after_redis_db_host_id,

    password='password_of_redis_db'
)

In [ ]:
r.ping()

In [20]:
import cv2
from insightface.app import FaceAnalysis

In [ ]:
faceapp=FaceAnalysis(name='buffalo_sc',
                     root='insightface_model',
                     )
faceapp.prepare(ctx_id=0, det_size=(640,640),det_thresh=0.5)

In [ ]:
person_name=input("enter your name: ")

trials=3
for i in range(trials):
    role=input(""" 
               Please choose
               1.Student
               2.Teacher
               
               enter no between either 1 or 2:
               """)
    if role in ('1','2'):
        if (role=='1'):
            role='Student'
        elif (role=='2'):
            role='Teacher'
        break
    else:
        print('please enter a valid no')
        if (i==2):
            print("max no of tries reached")
            role='Unknown'
            break
key=person_name+'@'+role
print('Your name= ',person_name)
print('Your role',role)
print('Key=',key)


    

In [ ]:
cap=cv2.VideoCapture(0)
face_embeddings=[]
sample=0
while True:
    ret,frame=cap.read()
    if ret == False:
        print('Unable to read Camera')
        break
    results=faceapp.get(frame,max_num=1)
    for res in results:
        sample+=1
        x1,y1,x2,y2=res['bbox'].astype(int)
        cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,0),1)
        embeddings=res['embedding']
        face_embeddings.append(embeddings)
    if sample>=200:
            break 
    cv2.namedWindow("Frame", cv2.WINDOW_GUI_EXPANDED)
    cv2.imshow('Frame',frame)
    if cv2.waitKey(1)==ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
len(face_embeddings)

In [ ]:
x_mean=np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

In [ ]:
x_mean_bytes=x_mean.tobytes()
r.hset(name='academy:register', key=key, value=x_mean_bytes)